# Metaheuristics - Flowshop problemm

### About this notebook
This notebook contains a hands-on the flowshop problem. We will focus on implementing some of the most known metaheuristic in order to solve this problem.

### Used ressources
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)
- [Implement Simulated annealing in Python](https://medium.com/swlh/how-to-implement-simulated-annealing-algorithm-in-python-ab196c2f56a0)

In [22]:
import numpy as np
import matplotlib as plt
import itertools
import time
import pandas as pd
import math
import random

In [2]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [4]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(15,10), low=5, high=120)
permutation = np.random.permutation(10).tolist()
print(rnd_data, "\n")
print('Initial solution:', permutation, "\n")

Cmax = evaluate_sequence(permutation, rnd_data)
print(f'Makespan: {Cmax}')

[[ 57  27  82  63  32  83  83 112  12 109]
 [ 66 108  74  12  27  65 114  82  46  26]
 [ 52  33  72  27 108  94  36 113  78  58]
 [109  84  96  86  68 104  19   8  24  74]
 [ 67  12   5  53  76  94  86  20  37  55]
 [  6 108  63  11  24 113  80  51  17  37]
 [ 59  71  49  39  16  20  19  55  71  79]
 [117  28  39  91  57  79 109  58  37 111]
 [ 53  32  59  38  39  41  39 117  70   8]
 [  6  94 110  90  12  17  17  94  22  75]
 [ 62  95  12   7  75  94  10  73 101  16]
 [ 49  70  30  49  16  31  52  14 113  96]
 [ 55  99  25  82  49  14  42  97  87  93]
 [ 76  92  42  77  31  40  77  15  76  56]
 [ 75  98  27  76  70  57  33  74 100  58]] 

Initial solution: [2, 3, 0, 5, 9, 1, 8, 6, 4, 7] 

Makespan: 1607.0


## Simulated annealing

In [17]:
def get_neighbor(solution):
    i = np.random.choice(list(solution))
    k = np.random.choice(list(solution))
    # Generating two different random positions
    while (i == k):
        k = np.random.choice(list(solution))
    # Switch between job i and job k in the given sequence
    neighbor = solution.copy()
    neighbor[i] = solution[k]
    neighbor[k] = solution[i]
    return neighbor

In [33]:
def simulated_annealing(initial_solution, processing_times, initial_temp=90, final_temp=1, alpha=0.1):
    current_temp = initial_temp
    current_solution = initial_solution.copy()
    current_cost = evaluate_sequence(initial_solution, processing_times)
    while current_temp > final_temp:
        neighbor = get_neighbor(current_solution)
        cost_diff = current_cost - evaluate_sequence(neighbor, processing_times)
        if cost_diff > 0:
            current_solution = neighbor
            current_cost = evaluate_sequence(current_solution, processing_times)
        else:
            if random.uniform(0, 1) < math.exp(-cost_diff / current_temp):
                solution = neighbor
        current_temp -= alpha
    return current_solution, current_cost

In [39]:
start_time = time.time()
best_solution, best_solution_length = simulated_annealing(permutation, rnd_data)
elapsed_time = time.time() - start_time

print("Best solution found: ", best_solution)
print("Makespan: ", evaluate_sequence(best_solution, rnd_data))
print("Elapsed time:", elapsed_time, "seconds")

Best solution found:  [2, 4, 9, 0, 5, 7, 3, 6, 8, 1]
Makespan:  1298.0
Elapsed time: 0.5146279335021973 seconds


## Tabu Search

## Genetic Algorithm